## Po poklicih (brez psihiatra)

In [33]:
return(data)

Job,Min,Max,Avg,SD,Group
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
Psiholog – delo z osebami s psihično motnjo,1952.08,2889.57,2420.825,468.745,psiholog
Psiholog II,1877.01,2778.42,2327.715,450.705,psiholog
Psiholog III,1735.39,2568.81,2152.100,416.710,psiholog
Specializant klinične psihologije,2195.84,3250.36,2723.100,527.260,klin_psiholog
Klinični psiholog specialist I,2889.57,4277.26,3583.415,693.845,klin_psiholog
Klinični psiholog specialist II,2778.42,4112.74,3445.580,667.160,klin_psiholog
Klinični psiholog specialist III,2470.01,3656.22,3063.115,593.105,klin_psiholog
Delovni terapevt v intenzivnih oddelkih psihiatrije,1804.80,2671.56,2238.180,433.380,delov_terapevt
Psiholog višji svetovalec,2375.00,2889.57,2632.285,257.285,psiholog


In [32]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA
library(FSA)       # For Dunn test after Kruskal-Wallis

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\psihoter_esej2.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E101", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:3, 9:11) ~ "psiholog",
    row_number() %in% c(4:7, 12:14) ~ "klin_psiholog",
    row_number() %in% c(24:100) ~ "psihoterapevt",
    TRUE ~ "delov_terapevt"
  )))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Plot and save function
save_plot <- function(metric, test_type, p_value) {
  plot <- ggplot(data, aes(x = Group, y = .data[[metric]])) +
    geom_boxplot() +
    labs(title = paste(metric, "by Group (", test_type, " p =", round(p_value, 15), ")"))
  
  ggsave(filename = paste0(metric, "_", test_type, ".png"), plot = plot, width = 8, height = 6, dpi = 300)
}

# Perform ANOVA if normality and homogeneity assumptions are met
metrics <- c("Min", "Max", "Avg", "SD")
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  for (metric in metrics) {
    # Perform ANOVA and Tukey post-hoc tests
    anova_res <- aov(reformulate("Group", metric), data = data)
    posthoc_res <- TukeyHSD(anova_res)
    
    # Print and plot ANOVA results
    print(summary(anova_res))
    print(posthoc_res)
    save_plot(metric, "ANOVA", summary(anova_res)[[1]][["Pr(>F)"]][1])
  }
} else {
  # Perform Kruskal-Wallis and Dunn post-hoc tests if assumptions are not met
  for (metric in metrics) {
    kw_res <- kruskal.test(reformulate("Group", metric), data = data)
    dunn_res <- dunnTest(data[[metric]], data$Group, method = "bonferroni")
    
    # Print and plot Kruskal-Wallis results
    print(kw_res)
    print(dunn_res$res)
    save_plot(metric, "Kruskal_Wallis_poklici", kw_res$p.value)
  }
}



	Kruskal-Wallis rank sum test

data:  Min by Group
Kruskal-Wallis chi-squared = 52.82, df = 3, p-value = 2.003e-11

                      Comparison           Z      P.unadj        P.adj
1 delov_terapevt - klin_psiholog -0.83178273 4.055316e-01 1.000000e+00
2      delov_terapevt - psiholog -0.03952774 9.684696e-01 1.000000e+00
3       klin_psiholog - psiholog  0.70009153 4.838701e-01 1.000000e+00
4 delov_terapevt - psihoterapevt  4.71226414 2.449796e-06 1.469877e-05
5  klin_psiholog - psihoterapevt  5.05068952 4.402181e-07 2.641309e-06
6       psiholog - psihoterapevt  3.78518150 1.535964e-04 9.215786e-04

	Kruskal-Wallis rank sum test

data:  Max by Group
Kruskal-Wallis chi-squared = 54.98, df = 3, p-value = 6.933e-12

                      Comparison          Z      P.unadj        P.adj
1 delov_terapevt - klin_psiholog -0.8601877 3.896856e-01 1.000000e+00
2      delov_terapevt - psiholog -0.1852163 8.530594e-01 1.000000e+00
3       klin_psiholog - psiholog  0.5900255 5.551736e-01 1.

## S psihoterapevti in psihiatri

In [ ]:
# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\psihoter_esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E102", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:9) ~ "zdravstvo",
    row_number() %in% c(10:24) ~ "soc_varstvo",
    TRUE ~ "psihoterapija"
  )))

return(data)

In [ ]:
kruskal_test <- kruskal.test(Avg ~ Group, data = data)
print(kruskal_test)

library(FSA)
dunn_test_results <- dunnTest(Avg ~ Group, data = data, method = "bonferroni")
print(dunn_test_results)

In [29]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA
library(FSA)       # For Dunn test after Kruskal-Wallis

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\psihoter_esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E102", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = factor(case_when(
    row_number() %in% c(1:9) ~ "zdravstvo",
    row_number() %in% c(10:24) ~ "soc_varstvo",
    TRUE ~ "psihoterapija"
  )))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Plot and save function
save_plot <- function(metric, test_type, p_value) {
  plot <- ggplot(data, aes(x = Group, y = .data[[metric]])) +
    geom_boxplot() +
    labs(title = paste(metric, "by Group (", test_type, " p =", round(p_value, 15), ")"))
  
  ggsave(filename = paste0(metric, "_", test_type, ".png"), plot = plot, width = 8, height = 6, dpi = 300)
}

# Perform ANOVA if normality and homogeneity assumptions are met
metrics <- c("Min", "Max", "Avg", "SD")
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  for (metric in metrics) {
    # Perform ANOVA and Tukey post-hoc tests
    anova_res <- aov(reformulate("Group", metric), data = data)
    posthoc_res <- TukeyHSD(anova_res)
    
    # Print and plot ANOVA results
    print(summary(anova_res))
    print(posthoc_res)
    save_plot(metric, "ANOVA", summary(anova_res)[[1]][["Pr(>F)"]][1])
  }
} else {
  # Perform Kruskal-Wallis and Dunn post-hoc tests if assumptions are not met
  for (metric in metrics) {
    kw_res <- kruskal.test(reformulate("Group", metric), data = data)
    dunn_res <- dunnTest(data[[metric]], data$Group, method = "bonferroni")
    
    # Print and plot Kruskal-Wallis results
    print(kw_res)
    print(dunn_res$res)
    save_plot(metric, "Kruskal_Wallis", kw_res$p.value)
  }
}



	Kruskal-Wallis rank sum test

data:  Min by Group
Kruskal-Wallis chi-squared = 53.862, df = 2, p-value = 2.014e-12

                   Comparison          Z      P.unadj        P.adj
1 psihoterapija - soc_varstvo -5.9431011 2.796797e-09 8.390392e-09
2   psihoterapija - zdravstvo -5.0453901 4.525971e-07 1.357791e-06
3     soc_varstvo - zdravstvo -0.2372836 8.124368e-01 1.000000e+00

	Kruskal-Wallis rank sum test

data:  Max by Group
Kruskal-Wallis chi-squared = 56.35, df = 2, p-value = 5.804e-13

                   Comparison          Z      P.unadj        P.adj
1 psihoterapija - soc_varstvo  6.5444428 5.971763e-11 1.791529e-10
2   psihoterapija - zdravstvo  4.5003777 6.783282e-06 2.034985e-05
3     soc_varstvo - zdravstvo -0.6205897 5.348696e-01 1.000000e+00

	Kruskal-Wallis rank sum test

data:  Avg by Group
Kruskal-Wallis chi-squared = 48.361, df = 2, p-value = 3.152e-11

                   Comparison          Z      P.unadj        P.adj
1 psihoterapija - soc_varstvo  6.2086600 5.3

## Brez psihiatra

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)


# Define groups for analysis
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esejbrez.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E24", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 9 ~ "zdravstvo",
    row_number() > 9 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() %in% c(1:3, 9:11) ~ "psiholog",
    row_number() %in% c(4:7, 12:14) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Display the modified data
return(data)


In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esejbrez.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E24", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 8 ~ "zdravstvo",
    row_number() > 8 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() %in% c(1:3, 9:11) ~ "psiholog",
    row_number() %in% c(4:7, 12:14) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Function to save plots with a specified filename
save_plot <- function(plot, filename, width = 8, height = 6, dpi = 300) {
  ggsave(filename = filename, plot = plot, width = width, height = height, dpi = dpi)
}

# Perform t-tests and ANOVA if criteria met (normality and variance)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)

  # Post-hoc tests for ANOVA
  posthoc_min <- TukeyHSD(anova_min)
  posthoc_max <- TukeyHSD(anova_max)
  posthoc_avg <- TukeyHSD(anova_avg)
  posthoc_sd <- TukeyHSD(anova_sd)

  # Print ANOVA and post-hoc results
  print(summary(anova_min)); print(posthoc_min)
  print(summary(anova_max)); print(posthoc_max)
  print(summary(anova_avg)); print(posthoc_avg)
  print(summary(anova_sd)); print(posthoc_sd)

  # List of plots with titles
  plots <- list(
    ggplot(data, aes(x = Group, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by zdravstvo and soc. varstvo_brez (p =", round(t_min$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by zdravstvo and soc. varstvo_brez (p =", round(t_max$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by zdravstvo and soc. varstvo_brez (p =", round(t_avg$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by zdravstvo and soc. varstvo_brez (p =", round(t_sd$p.value, 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by poklic_brez (ANOVA p =", round(summary(anova_min)[[1]][["Pr(>F)"]][1], 5), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by poklic_brez (ANOVA p =", round(summary(anova_max)[[1]][["Pr(>F)"]][1], 5), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by poklic_brez (ANOVA p =", round(summary(anova_avg)[[1]][["Pr(>F)"]][1], 5), ")")),
    
    ggplot(data, aes(x = SubGroup, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by poklic_brez (ANOVA p =", round(summary(anova_sd)[[1]][["Pr(>F)"]][1], 5), ")"))
  )
  
  # Filenames for each plot
  plot_filenames <- c("Min_by_zdravstvo_and_socvarstvo_brez.png", "Max_by_zdravstvo_and_socvarstvo_brez.png", "Avg_by_zdravstvo_and_socvarstvo_brez.png", "SD_by_zdravstvo_and_socvarstvo_brez.png",
                      "Min_by_poklic_brez.png", "Max_by_poklic_brez.png", "Avg_by_poklic_brez.png", "SD_by_poklic_brez.png")
  
  # Save each plot with corresponding filename
  for (i in seq_along(plots)) {
    save_plot(plots[[i]], plot_filenames[i])
  }
}


## S psihiatri

In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E25", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define groups for analysis
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 9 ~ "zdravstvo",
    row_number() > 9 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() == 1 ~ "psihiater",
    row_number() %in% c(2:4, 10:12) ~ "psiholog",
    row_number() %in% c(5:8, 13:15) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>%
  group_by(Group) %>%
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Function to save plots with a specified filename
save_plot <- function(plot, filename, width = 8, height = 6, dpi = 300) {
  ggsave(filename = filename, plot = plot, width = width, height = height, dpi = dpi)
}

# Perform t-tests and ANOVA if criteria met (normality and variance)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)

  # Post-hoc tests for ANOVA
  posthoc_min <- TukeyHSD(anova_min)
  posthoc_max <- TukeyHSD(anova_max)
  posthoc_avg <- TukeyHSD(anova_avg)
  posthoc_sd <- TukeyHSD(anova_sd)

  # Print ANOVA and post-hoc results
  print(summary(anova_min)); print(posthoc_min)
  print(summary(anova_max)); print(posthoc_max)
  print(summary(anova_avg)); print(posthoc_avg)
  print(summary(anova_sd)); print(posthoc_sd)

  # List of plots with titles
  plots <- list(
    ggplot(data, aes(x = Group, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by zdravstvo and soc. varstvo (p =", round(t_min$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by zdravstvo and soc. varstvo (p =", round(t_max$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by zdravstvo and soc. varstvo (p =", round(t_avg$p.value, 4), ")")),
    
    ggplot(data, aes(x = Group, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by zdravstvo and soc. varstvo (p =", round(t_sd$p.value, 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Min)) + 
      geom_boxplot() + 
      labs(title = paste("Min by poklic (ANOVA p =", round(summary(anova_min)[[1]][["Pr(>F)"]][1], 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Max)) + 
      geom_boxplot() + 
      labs(title = paste("Max by poklic (ANOVA p =", round(summary(anova_max)[[1]][["Pr(>F)"]][1], 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = Avg)) + 
      geom_boxplot() + 
      labs(title = paste("Avg by poklic (ANOVA p =", round(summary(anova_avg)[[1]][["Pr(>F)"]][1], 4), ")")),
    
    ggplot(data, aes(x = SubGroup, y = SD)) + 
      geom_boxplot() + 
      labs(title = paste("SD by poklic (ANOVA p =", round(summary(anova_sd)[[1]][["Pr(>F)"]][1], 4), ")"))
  )
  
  # Filenames for each plot
  plot_filenames <- c("Min_by_zdravstvo_and_socvarstvo.png", "Max_by_zdravstvo_and_socvarstvo.png", "Avg_by_zdravstvo_and_socvarstvo.png", "SD_by_zdravstvo_and_socvarstvo.png",
                      "Min_by_poklic.png", "Max_by_poklic.png", "Avg_by_poklic.png", "SD_by_poklic.png")
  
  # Save each plot with corresponding filename
  for (i in seq_along(plots)) {
    save_plot(plots[[i]], plot_filenames[i])
  }
}


In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)
library(car)       # For Levene's test
library(multcomp)  # For post-hoc tests after ANOVA

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E25", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define the groups for analysis based on row ranges
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 10 ~ "zdravstvo",
    row_number() > 10 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() == 2 ~ "psihiater",
    row_number() %in% c(3:5, 11:13) ~ "psiholog",
    row_number() %in% c(6:9, 14:16) ~ "klin_psiholog",
    TRUE ~ "delov_terapevt"
  ))

# Check criteria for t-test and ANOVA
# Check normality and variance equality (Shapiro-Wilk for normality and Levene's test for variance)

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>% 
  group_by(Group) %>% 
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Perform t-tests and ANOVA if criteria met (normality p > 0.05 and homogeneity p > 0.05)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons (A, B, C, D)
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)
  
  # Post-hoc tests for ANOVA
  posthoc_min <- TukeyHSD(anova_min)
  posthoc_max <- TukeyHSD(anova_max)
  posthoc_avg <- TukeyHSD(anova_avg)
  posthoc_sd <- TukeyHSD(anova_sd)
  
  # Display ANOVA and post-hoc results
  print(summary(anova_min))
  print(posthoc_min)
  print(summary(anova_max))
  print(posthoc_max)
  print(summary(anova_avg))
  print(posthoc_avg)
  print(summary(anova_sd))
  print(posthoc_sd)
  
  # Adding p-values to group boxplots
  ggplot(data, aes(x = Group, y = Min)) + 
    geom_boxplot() + 
    labs(title = paste("Min by zdravstvo and soc. varstvo (p =", round(t_min$p.value, 4), ")"))

  ggplot(data, aes(x = Group, y = Max)) + 
    geom_boxplot() + 
    labs(title = paste("Max by zdravstvo and soc. varstvo (p =", round(t_max$p.value, 4), ")"))

  ggplot(data, aes(x = Group, y = Avg)) + 
    geom_boxplot() + 
    labs(title = paste("Avg by zdravstvo and soc. varstvo (p =", round(t_avg$p.value, 4), ")"))

  ggplot(data, aes(x = Group, y = SD)) + 
    geom_boxplot() + 
    labs(title = paste("SD by zdravstvo and soc. varstvo (p =", round(t_sd$p.value, 4), ")"))

  # Boxplots for SubGroup comparisons (with ANOVA)
  ggplot(data, aes(x = SubGroup, y = Min)) + 
    geom_boxplot() + 
    labs(title = paste("Min by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_min)[[1]][["Pr(>F)"]][1], 4), ")"))

  ggplot(data, aes(x = SubGroup, y = Max)) + 
    geom_boxplot() + 
    labs(title = paste("Max by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_max)[[1]][["Pr(>F)"]][1], 4), ")"))

  ggplot(data, aes(x = SubGroup, y = Avg)) + 
    geom_boxplot() + 
    labs(title = paste("Avg by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_avg)[[1]][["Pr(>F)"]][1], 4), ")"))

  ggplot(data, aes(x = SubGroup, y = SD)) + 
    geom_boxplot() + 
    labs(title = paste("SD by psihiater, psiholog, klin_psih, delov_terapevt (ANOVA p =", round(summary(anova_sd)[[1]][["Pr(>F)"]][1], 4), ")"))
}


In [ ]:
# Load necessary libraries
library(readxl)
library(dplyr)
library(ggplot2)

# Import the .xlsx file
file_path <- "C:\\Users\\Neza\\Documents\\programi\\Esej.xlsx" # Replace with your actual file path
data <- read_excel(file_path, range = "A2:E25", col_names = c("Job", "Min", "Max", "Avg", "SD"))

# Define the groups for analysis based on row ranges
data <- data %>%
  mutate(Group = case_when(
    row_number() <= 10 ~ "zdravstvo",
    row_number() > 10 ~ "soc_varstvo"
  ),
  SubGroup = case_when(
    row_number() == 2 ~ "psihiater",
    row_number() %in% c(3:5, 11:13) ~ "psiholog",
    row_number() %in% c(6:9, 14:16) ~ "klin_psiholog",
    TRUE ~ "delov_terap"
  ))

# Check criteria for t-test and ANOVA
# Check normality and variance equality (assuming Shapiro-Wilk for normality and Levene's test for variance)

# Shapiro-Wilk Test for normality
shapiro_test_results <- data %>% 
  group_by(Group) %>% 
  summarise(p_value = shapiro.test(Avg)$p.value)

# Levene's test for homogeneity of variances (install 'car' package if needed)
library(car)
levene_test_results <- leveneTest(Avg ~ Group, data = data)

# Perform t-tests and ANOVA if criteria met (normality p > 0.05 and homogeneity p > 0.05)
if (all(shapiro_test_results$p_value > 0.05) & levene_test_results$`Pr(>F)`[1] > 0.05) {
  # T-tests for Group comparisons
  t_min <- t.test(Min ~ Group, data = data)
  t_max <- t.test(Max ~ Group, data = data)
  t_avg <- t.test(Avg ~ Group, data = data)
  t_sd <- t.test(SD ~ Group, data = data)

  # ANOVA for SubGroup comparisons (A, B, C, D)
  anova_min <- aov(Min ~ SubGroup, data = data)
  anova_max <- aov(Max ~ SubGroup, data = data)
  anova_avg <- aov(Avg ~ SubGroup, data = data)
  anova_sd <- aov(SD ~ SubGroup, data = data)

  # Display results
  print(summary(anova_min))
  print(summary(anova_max))
  print(summary(anova_avg))
  print(summary(anova_sd))
}

# Plotting the results
# Boxplots for each measure by Group and SubGroup
ggplot(data, aes(x = Group, y = Min)) + geom_boxplot() + labs(title = "Min by zdravstvo and soc. varstvo")
ggplot(data, aes(x = Group, y = Max)) + geom_boxplot() + labs(title = "Max by zdravstvo and soc. varstvo")
ggplot(data, aes(x = Group, y = Avg)) + geom_boxplot() + labs(title = "Avg by zdravstvo and soc. varstvo")
ggplot(data, aes(x = Group, y = SD)) + geom_boxplot() + labs(title = "SD by zdravstvo and soc. varstvo")

ggplot(data, aes(x = SubGroup, y = Min)) + geom_boxplot() + labs(title = "Min by psihiater, psiholog, klin_psih, delov_terapevt")
ggplot(data, aes(x = SubGroup, y = Max)) + geom_boxplot() + labs(title = "Max by psihiater, psiholog, klin_psih, delov_terapevt")
ggplot(data, aes(x = SubGroup, y = Avg)) + geom_boxplot() + labs(title = "Avg by psihiater, psiholog, klin_psih, delov_terapevt")
ggplot(data, aes(x = SubGroup, y = SD)) + geom_boxplot() + labs(title = "SD by psihiater, psiholog, klin_psih, delov_terapevt")
